In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import argparse
import time
import sys
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt

In [2]:
sys.path.append('/home/ziniu.wzn/FSPN/fspn')
from Data_prepare.schemas import gen_stats_light_schema
from Data_prepare.query_prepare_BayesCard import prepare_join_queries
schema = gen_stats_light_schema("/home/ziniu.wzn/stats")
hdf_path = "/home/ziniu.wzn/stats/gen_single_light"
meta_data_path = hdf_path + '/meta_data.pkl'

ensemble_location = "/home/ziniu.wzn/stats/FSPN_ensemble/"
#ensemble_location = "/home/yuxing.hyx/dataset/imdb-benchmark/spn_ensembles/ensemble_join_3_budget_5_10000000.pkl"
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_sub_queries.sql"
#query_filename = "/home/ziniu.wzn/deepdb-public/benchmarks/job-light/sql/job_light_queries.sql"
pairwise_rdc_path = "/home/yuxing.hyx/repository/imdb-benchmark/spn_ensembles/pairwise_rdc.pkl"
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path, query_filename, 
                                      join_3_rdc_based=True, true_card_exist=True)

model_9_2060.pkl


/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/home/ziniu.wzn/anaconda3/envs/deepdb/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.

model_2_1964.pkl
model_7_1037.pkl
model_1_3156.pkl
model_0_3069.pkl
model_3_5008.pkl
model_8_1107.pkl
model_4_2538.pkl
model_5_1755.pkl
model_6_2202.pkl


In [3]:
sys.path.append('/home/ziniu.wzn/BayesCard')
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path="/home/ziniu.wzn/stats/BayesCard_ensemble/"):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)

8_CLT.pkl
0_CLT.pkl
9_CLT.pkl
7_CLT.pkl
6_CLT.pkl
2_CLT.pkl
4_CLT.pkl
1_CLT.pkl
3_CLT.pkl
5_CLT.pkl


In [4]:
with open(query_filename, "rb") as f:
    real_query = f.readlines()

queries = bn_ensemble.parse_query_all(parsed_queries)


In [5]:
latency = []
q_errors = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"predicting query no {i}: {real_query[i]} \n")
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

predicting query no 0: b"SELECT COUNT(*) FROM posts as p, comments as c WHERE c.PostId= p.Id AND p.CreationDate>='2010-07-20 00:52:13'::timestamp;||0||173949\n" 

true cardinality 173949, predicted 173379.2315681988 with q-error 1.003286255375847
predicting query no 1: b'SELECT COUNT(*) FROM users as u, badges as b WHERE b.UserId= u.Id AND u.UpVotes>=0;||1||79851\n' 

true cardinality 79851, predicted 84477.1438941475 with q-error 1.0579347020594294
predicting query no 2: b'SELECT COUNT(*) FROM users as u, posts as p WHERE p.OwnerUserId= u.Id AND u.Views>=0;||2||90584\n' 

true cardinality 90584, predicted 94899.49978322556 with q-error 1.0476408613356174
predicting query no 3: b"SELECT COUNT(*) FROM badges as b, comments as c, users as u WHERE c.UserId = u.Id AND b.UserId = u.Id AND b.Date<='2014-09-11 14:33:06'::timestamp AND c.Score=0;||3||10220614\n" 

true cardinality 10220614, predicted 10110734.523138978 with q-error 1.0108676057718218
predicting query no 4: b'SELECT COUNT(*) FR

predicting query no 27: b"SELECT COUNT(*) FROM postHistory as ph, comments as c, users as u WHERE ph.UserId = u.Id AND c.UserId = u.Id AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-21 03:54:23'::timestamp;||14||108828592\n" 

true cardinality 108828592, predicted 107504493.41714242 with q-error 1.0123166812918207
predicting query no 28: b'SELECT COUNT(*) FROM users as u, comments as c WHERE c.UserId = u.Id AND u.Reputation>=1;||14||171470\n' 

true cardinality 171470, predicted 182048.11825059872 with q-error 1.061690781189705
predicting query no 29: b"SELECT COUNT(*) FROM users as u, postHistory as ph WHERE ph.UserId = u.Id AND u.Reputation>=1 AND ph.PostHistoryTypeId=5 AND ph.CreationDate<='2014-08-21 03:54:23'::timestamp;||14||66958\n" 

true cardinality 66958, predicted 64541.12472211172 with q-error 1.0374470585738067
predicting query no 30: b"SELECT COUNT(*) FROM users as u, comments as c, postHistory as ph WHERE ph.UserId = u.Id AND c.UserId = u.Id AND u.Reputation>=1

predicting query no 67: b"SELECT COUNT(*) FROM users as u, posts as p WHERE p.OwnerUserId = u.Id AND u.Views>=0 AND u.DownVotes>=0 AND p.CreationDate>='2010-09-05 11:37:47'::timestamp AND p.CreationDate<='2014-09-12 08:58:41'::timestamp;||24||88358\n" 

true cardinality 88358, predicted 89843.38361236699 with q-error 1.0168109691523912
predicting query no 68: b"SELECT COUNT(*) FROM users as u, postHistory as ph, posts as p WHERE ph.UserId = u.Id AND p.OwnerUserId = u.Id AND u.Views>=0 AND u.DownVotes>=0 AND p.CreationDate>='2010-09-05 11:37:47'::timestamp AND p.CreationDate<='2014-09-12 08:58:41'::timestamp;||24||53002352\n" 

true cardinality 53002352, predicted 52948813.13717976 with q-error 1.0010111437754332
predicting query no 69: b"SELECT COUNT(*) FROM posts as p, postHistory as ph WHERE ph.PostId = p.Id AND p.CreationDate>='2010-07-26 19:26:37'::timestamp AND p.CreationDate<='2014-08-22 14:43:39'::timestamp AND ph.CreationDate<='2014-08-17 21:24:11'::timestamp;||25||291926\n" 



predicting query no 104: b"SELECT COUNT(*) FROM votes as v, posts as p, postLinks as pl WHERE v.PostId = p.Id AND pl.PostId = p.Id AND v.VoteTypeId=2 AND v.CreationDate<='2014-09-12 00:00:00'::timestamp AND p.Score>=-3;||32||45455\n" 

true cardinality 45455, predicted 78104.77634511306 with q-error 1.7182878967135202
predicting query no 105: b"SELECT COUNT(*) FROM votes as v, comments as c, posts as p, postLinks as pl WHERE pl.PostId = p.Id AND v.PostId = p.Id AND c.PostId = p.Id AND v.VoteTypeId=2 AND v.CreationDate<='2014-09-12 00:00:00'::timestamp AND c.CreationDate>='2010-08-02 23:52:10'::timestamp AND p.Score>=-3;||32||176191\n" 

true cardinality 176191, predicted 194789.9908766692 with q-error 1.1055615262792606
predicting query no 106: b"SELECT COUNT(*) FROM posts as p, comments as c WHERE p.Id = c.PostId AND c.CreationDate>='2010-07-21 11:05:37'::timestamp AND c.CreationDate<='2014-08-25 17:59:25'::timestamp;||33||170249\n" 

true cardinality 170249, predicted 171835.87408319

predicting query no 136: b"SELECT COUNT(*) FROM posts as p, comments as c, users as u WHERE u.Id = p.OwnerUserId AND u.Id = c.UserId AND p.CreationDate>='2010-07-27 01:51:15'::timestamp AND c.Score=0 AND c.CreationDate<='2014-09-13 20:12:15'::timestamp;||36||36980622\n" 

true cardinality 36980622, predicted 40277244.98236291 with q-error 1.0891446061227124
predicting query no 137: b"SELECT COUNT(*) FROM votes as v, comments as c, users as u WHERE u.Id = c.UserId AND u.Id = v.UserId AND v.BountyAmount<=50 AND v.CreationDate<='2014-09-12 00:00:00'::timestamp AND c.Score=0 AND c.CreationDate<='2014-09-13 20:12:15'::timestamp;||36||242937\n" 

true cardinality 242937, predicted 239879.7313873953 with q-error 1.0127450059866347
predicting query no 138: b"SELECT COUNT(*) FROM users as u, comments as c WHERE u.Id = c.UserId AND u.UpVotes>=0 AND u.UpVotes<=12 AND u.CreationDate>='2010-07-19 19:09:39'::timestamp AND c.Score=0 AND c.CreationDate<='2014-09-13 20:12:15'::timestamp;||36||42543\n" 

...

predicting query no 1786: b'SELECT COUNT(*) FROM users as u, comments as c, posts as p, postHistory as ph, badges as b WHERE u.Id = p.OwnerUserId AND p.Id = c.PostId AND u.Id = b.UserId AND p.Id = ph.PostId AND u.Views<=86 AND u.DownVotes>=0 AND u.DownVotes<=1 AND u.UpVotes<=6 AND p.PostTypeId=1 AND p.ViewCount<=4159 AND p.CommentCount>=0 AND p.CommentCount<=12 AND ph.PostHistoryTypeId=2;||109||175748\n' 

true cardinality 175748, predicted 14267.473948336092 with q-error 12.318088025701016
predicting query no 1787: b'SELECT COUNT(*) FROM users as u, comments as c, posts as p, votes as v, badges as b WHERE p.Id = v.PostId AND u.Id = p.OwnerUserId AND p.Id = c.PostId AND u.Id = b.UserId AND u.Views<=86 AND u.DownVotes>=0 AND u.DownVotes<=1 AND u.UpVotes<=6 AND p.PostTypeId=1 AND p.ViewCount<=4159 AND p.CommentCount>=0 AND p.CommentCount<=12 AND v.VoteTypeId=2;||109||434967\n' 

true cardinality 434967, predicted 42833.43258319434 with q-error 10.15484806535582
predicting query no 1788: 

predicting query no 1812: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p, postHistory as ph WHERE u.Id = p.OwnerUserId AND p.Id = ph.PostId AND p.Id = c.PostId AND u.Reputation>=1 AND u.Views>=0 AND u.CreationDate>='2010-08-19 06:26:34'::timestamp AND u.CreationDate<='2014-09-11 05:22:26'::timestamp AND p.ViewCount>=0 AND p.AnswerCount>=0 AND p.AnswerCount<=7 AND p.FavoriteCount>=0 AND p.FavoriteCount<=17;||110||175892\n" 

true cardinality 175892, predicted 92713.61317600185 with q-error 1.8971539774433923
predicting query no 1813: b"SELECT COUNT(*) FROM badges as b, comments as c, posts as p, votes as v, users as u WHERE p.Id = v.PostId AND u.Id = p.OwnerUserId AND p.Id = c.PostId AND u.Id = b.UserId AND b.Date>='2010-08-01 02:54:53'::timestamp AND p.ViewCount>=0 AND p.AnswerCount>=0 AND p.AnswerCount<=7 AND p.FavoriteCount>=0 AND p.FavoriteCount<=17 AND v.VoteTypeId=5;||110||1440551\n" 

true cardinality 1440551, predicted 22811.43705389364 with q-error 63.150383581560

predicting query no 1834: b'SELECT COUNT(*) FROM users as u, badges as b WHERE u.Id = b.UserId;||111||79851\n' 

true cardinality 79851, predicted 84820.36276535377 with q-error 1.0622329434240494
predicting query no 1835: b'SELECT COUNT(*) FROM postHistory as ph, comments as c, posts as p WHERE p.Id = ph.PostId AND p.Id = c.PostId AND p.PostTypeId=1 AND p.CommentCount>=0 AND p.CommentCount<=12 AND p.FavoriteCount>=0;||111||183554\n' 

true cardinality 183554, predicted 180490.17081330015 with q-error 1.0169750472997727
predicting query no 1836: b'SELECT COUNT(*) FROM votes as v, comments as c, posts as p WHERE p.Id = v.PostId AND p.Id = c.PostId AND v.BountyAmount<=50 AND p.PostTypeId=1 AND p.CommentCount>=0 AND p.CommentCount<=12 AND p.FavoriteCount>=0;||111||1557\n' 

true cardinality 1557, predicted 984.3655171247681 with q-error 1.5817295231428252
predicting query no 1837: b'SELECT COUNT(*) FROM badges as b, comments as c, posts as p, users as u WHERE u.Id = p.OwnerUserId AND p.Id

predicting query no 1859: b'SELECT COUNT(*) FROM users as u, comments as c, posts as p, votes as v, badges as b WHERE p.Id = v.PostId AND u.Id = p.OwnerUserId AND p.Id = c.PostId AND u.Id = b.UserId AND p.PostTypeId=1 AND p.CommentCount>=0 AND p.CommentCount<=12 AND p.FavoriteCount>=0 AND v.BountyAmount<=50;||111||48222\n' 

true cardinality 48222, predicted 967.9533877353579 with q-error 49.81851462168143
predicting query no 1860: b'SELECT COUNT(*) FROM users as u, posts as p, postHistory as ph, votes as v, badges as b WHERE p.Id = v.PostId AND u.Id = p.OwnerUserId AND p.Id = ph.PostId AND u.Id = b.UserId AND p.PostTypeId=1 AND p.CommentCount>=0 AND p.CommentCount<=12 AND p.FavoriteCount>=0 AND v.BountyAmount<=50;||111||109211\n' 

true cardinality 109211, predicted 2771.2393571535868 with q-error 39.4087214870438
predicting query no 1861: b'SELECT COUNT(*) FROM users as u, comments as c, posts as p, postHistory as ph, votes as v, badges as b WHERE u.Id = p.OwnerUserId AND u.Id = b.Us

predicting query no 1884: b'SELECT COUNT(*) FROM users as u, comments as c, posts as p, postHistory as ph WHERE u.Id = p.OwnerUserId AND p.Id = ph.PostId AND p.Id = c.PostId AND u.Reputation<=240 AND c.Score=0 AND p.Score<=21 AND p.AnswerCount<=3 AND p.FavoriteCount>=0;||112||80402\n' 

true cardinality 80402, predicted 92795.22040663574 with q-error 1.1541406980751192
predicting query no 1885: b"SELECT COUNT(*) FROM badges as b, comments as c, posts as p, votes as v, users as u WHERE p.Id = v.PostId AND u.Id = p.OwnerUserId AND p.Id = c.PostId AND u.Id = b.UserId AND b.Date<='2014-09-11 18:35:08'::timestamp AND c.Score=0 AND p.Score<=21 AND p.AnswerCount<=3 AND p.FavoriteCount>=0 AND v.CreationDate>='2010-07-19 00:00:00'::timestamp;||112||2662177\n" 

true cardinality 2662177, predicted 108768.92017142699 with q-error 24.4755302875512
predicting query no 1886: b"SELECT COUNT(*) FROM users as u, comments as c, posts as p, votes as v WHERE p.Id = v.PostId AND u.Id = p.OwnerUserId AND p.

In [ ]:
parsed_queries[1869]

In [ ]:
q_errors = np.asarray(q_errors)

In [ ]:
np.where(q_errors > 5000)

In [ ]:
q = queries[1869]

In [ ]:
print(len(q))
q

In [ ]:
real_query[1869]

In [ ]:
bn_ensemble.bns[1].table_name

In [ ]:
bn_ensemble.bns[0].

In [ ]:
parsed_queries[1869]

In [ ]:
p, _ = bn_ensemble.bns[1].query(q[1]["query"], n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
p

In [ ]:
p, _ = bn_ensemble.bns[1].expectation(q[1]["query"], e, n_distinct=q[1]["n_distinct"], return_prob=True)

In [ ]:
q[0]*p

In [ ]:
e = ['posts.mul_postHistory.PostId', 'posts.mul_comments.PostId_nn']

In [ ]:
bn_ensemble.bns[1].fanout_attr_inverse

In [3]:
pred = np.load("/home/ziniu.wzn/stats/est_results/bayesCard_join.npy")

In [6]:
query_filename = "/home/ziniu.wzn/deepdb-stats/benchmarks/stats/stats_CEB_sub_queries.sql"
with open(query_filename, "r") as f:
    queries = f.readlines()
true = [int(q.split("||")[-1]) for q in queries]
true = np.asarray(true)

In [10]:
q_errors = np.maximum(true/pred, pred/true)

In [11]:
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

q-error 50% percentile is 1.1822773172408918
q-error 90% percentile is 50.40486533860654
q-error 95% percentile is 95.17965061035127
q-error 99% percentile is 156.38902010629596
q-error 100% percentile is 8559.006677218465


In [15]:
pred2 = pred + np.random.normal(10, 100, size=2603)
pred2[pred2<=1] = 1

In [16]:
q_errors = np.maximum(true/pred2, pred2/true)
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")

q-error 50% percentile is 1.1857489744089196
q-error 90% percentile is 50.65523456585348
q-error 95% percentile is 95.2919946477845
q-error 99% percentile is 156.8525937595126
q-error 100% percentile is 8559.159897279067


In [17]:
np.save("/home/ziniu.wzn/stats/est_results/bayesCard_join_update", pred2)